# What is around each of top 100 world-class universities?

In [27]:
import pandas as pd
import numpy as np
import html5lib
import lxml
import re
import requests
import json
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import matplotlib.pyplot as plt
import folium
import time
import geocoder
from tqdm import tqdm
import sys
import linecache
pd.set_option('display.max_rows', None)

In [2]:
option = webdriver.ChromeOptions()
chrome_prefs = {}
option.experimental_options["prefs"] = chrome_prefs
option.add_argument('--headless')
option.add_argument('--no-sandbox')
option.add_argument('--disable-dev-shm-usage')
chrome_prefs["profile.default_content_settings"] = {"images": 2}
chrome_prefs["profile.managed_default_content_settings"] = {"images": 2}

In [3]:
url = 'https://www.topuniversities.com/university-rankings/world-university-rankings/2020'

In [4]:
tableCSSselector = '.quicktabs_main'
clickPerPageCSSselector = '#qs-rankings_length > label > span.jcf-select.jcf-unselectable > span.jcf-select-opener'
click100CSSselector = '#qs-rankings_length > label > span.jcf-select.jcf-unselectable.jcf-drop-active > div > div > span > span > ul > li:nth-child(3) > span'

In [5]:
def PrintException():
    exc_type, exc_obj, tb = sys.exc_info()
    f = tb.tb_frame
    lineno = tb.tb_lineno
    filename = f.f_code.co_filename
    linecache.checkcache(filename)
    line = linecache.getline(filename, lineno, f.f_globals)
    print('Exception in ({}, line {} "{}"): {}'.format(filename, lineno, line.strip(), exc_obj))

In [8]:
driver = webdriver.Chrome() #options = option)
driver.get(url)

wait = WebDriverWait(driver, 20)
wait.until(EC.visibility_of_all_elements_located((By.CSS_SELECTOR, tableCSSselector)))
print('ready ro scrape.')

driver.execute_script("window.scrollTo(0, 2100)")

time.sleep(1)

cursor = driver.find_element_by_css_selector(clickPerPageCSSselector)
cursor.click()
time.sleep(2)

cursor = driver.find_element_by_css_selector(click100CSSselector)
cursor.click()
time.sleep(5)

print('Top 100 institutions shown')

soup = BeautifulSoup(driver.page_source, 'html5lib')

QSurl = 'https://www.topuniversities.com'

uniName = []
uniLink = []
for i in soup.findAll('a', class_='title'):
    uniName.append(i.text)
    uniLink.append(i.get('href'))

mapCanvasXpath = '//*[@id="scene"]/div[3]/canvas'
whatsHereCSSselector = '#action-menu > ul > li:nth-child(3)'
LatLonCSSselector = '#reveal-card > div > div.widget-reveal-card-container > button.link-like.widget-reveal-card-lat-lng'

uniLocList = []
LatLonList = []

for i in tqdm(range(len(uniName))):
    try:
        urlUni = QSurl + uniLink[i]
        driver.get(urlUni)
    
        soupUni = BeautifulSoup(driver.page_source, 'html5lib')
    
        uniLocRaw = soupUni.select_one('.location')
        if uniLocRaw == None:
            locCSSselectorAltern = '#block-system-main > div > div.basic-pro > div.lead_section > header > div.container > div.title_info > div.location'
            uniLocRaw = soupUni.select_one(locCSSselectorAltern)

        uniLoc = re.findall('(?!\n).+', uniLocRaw.text)[0]
        uniLoc = uniLoc[:uniLoc.find(' View')]
    
#     urlUniLoc = uniLoc.find('a').get('href')
#     gmapAPI = urlUniLoc[urlUniLoc.find('//')+2:]
    
#     driver.get('https://' + gmapAPI)
    
#     wait = WebDriverWait(driver, 30)
#     wait.until(EC.visibility_of_all_elements_located((By.XPATH, mapCanvasXpath)))
    
#     action = ActionChains(driver)
#     action.move_to_element(driver.find_element_by_xpath(mapCanvasXpath)).perform()
#     action.context_click().perform()
#     time.sleep(2)
    
#     cursor = driver.find_element_by_css_selector(whatsHereCSSselector)
#     cursor.click()
#     time.sleep(2)
    
#     soupGmaps = BeautifulSoup(driver.page_source, 'html5lib')

#     LatLon = soupGmaps.select_one(LatLonCSSselector).get('aria-label')
#     LatLon = list(map(float, LatLon.split(', ')))
    
        uniLocList.append(uniLoc)
#     LatLonList.append(LatLon)
        
        print(str(i+1) + ': ' + uniName[i])
        time.sleep(1)
    except:
        print('Error occured when scraping ' + uniName[i])
        PrintException()
        print('Continue to the next iteration.')
        uniLocList.append('')
        continue
    
driver.close()

ready ro scrape.
Top 100 institutions shown


  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

1: Massachusetts Institute of Technology (MIT)


  1%|▊                                                                                 | 1/100 [00:05<08:56,  5.42s/it]

2: Stanford University


...

 98%|███████████████████████████████████████████████████████████████████████████████▍ | 98/100 [14:14<00:15,  7.99s/it]

99: KTH Royal Institute of Technology


 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [14:21<00:07,  7.79s/it]

100: University of St Andrews


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [14:28<00:00,  8.69s/it]


In [23]:
uniCountryList = []
uniAddressList = []
uniCityList = []

for j in uniLocList:
    try:
        uniCountry = re.findall('(?<=,\s)\w+(?:$|\s\w+$)', j)[0]
        uniAddress = re.findall('.+(?=,.+$)', j)[0]
        uniCity = re.findall('\w+(?=,\s\w+(?:$|\s\w+$))', j)[0]
    except:
        if 'Hong Kong' in j:
            uniCity = 'Hong Kong S.A.R.'
        elif '' in j:
            uniCountry = ''
            uniAddress = ''
            uniCity = ''
    
    if 'City' in uniCity:
        uniCity = 'Kyoto'
    elif 'York' in uniCity:
        uniCity = 'New York'
    elif 'Haven' in uniCity:
        uniCity = 'New Haven'
    elif 'Angeles' in uniCity:
        uniCity = 'Los Angeles'
    elif 'Jolla' in uniCity:
        uniCity = 'La Jolla'
    elif 'Lumpur' in uniCity:
        uniCity = 'Kuala Lumpur'
    elif 'Aires' in uniCity:
        uniCity = 'Buenos Aires'
    elif 'Hill' in uniCity:
        uniCity = 'Chapel Hill'
    elif 'Park' in uniCity:
        uniCity = 'University Park'
    elif 'Andrews' in uniCity:
        uniCity = 'St. Andrews'
    
    uniCountryList.append(uniCountry)
    uniAddressList.append(uniAddress)
    uniCityList.append(uniCity)

In [29]:
uniCity

'St. Andrews'

In [32]:
dataUni = [uniName, uniCityList, uniCountryList, uniLocList]
df = pd.DataFrame(dataUni).T
df.columns = ['Institution Name','City/Town','Country','Address']

In [33]:
df

,Institution Name,City/Town,Country,Address
0,Massachusetts Institute of Technology (MIT),Cambridge,United States,"77 Massachusetts Avenue Cambridge, United States"
1,Stanford University,Stanford,United States,"450 Serra Mall Stanford, United States"
2,Harvard University,Cambridge,United States,"1350 Massachusetts Avenue Cambridge, United St..."
3,University of Oxford,Oxford,United Kingdom,"Wellington Square Oxford, United Kingdom"
4,California Institute of Technology (Caltech),Pasadena,United States,"1200 East California Boulevard Pasadena, Unite..."
5,ETH Zurich - Swiss Federal Institute of Techno...,Zürich,Switzerland,"Raemistrasse 101 Zürich, Switzerland"
6,University of Cambridge,Cambridge,United Kingdom,"The Old Schools, Trinity Lane Cambridge, Unite..."
7,UCL,London,United Kingdom,"University College London, Gower Street London..."
8,Imperial College London,London,United Kingdom,"South Kensington London, United Kingdom"
9,University of Chicago,Chicago,United States,"Edward H. Levi Hall, 5801 South Ellis Avenue C..."


Get the full address of [Université PSL](https://www.timeshighereducation.com/world-university-rankings/paris-sciences-et-lettres-psl-research-university-paris) and [Sorbonne University](https://www.timeshighereducation.com/world-university-rankings/sorbonne-university) from Times Higher Education.

In [34]:
UPSLloc = '60 rue Mazarine, Paris, 75006, France'
SorbonneLoc = "21 Rue de l'Ecole-de-Medecine, Paris, 75006, France"

In [43]:
df.loc[df['Institution Name'] == 'Université PSL', ['City/Town','Country','Address']] = ['Paris','France',UPSLloc]
df.loc[df['Institution Name'] == 'Sorbonne University', ['City/Town','Country','Address']] = ['Paris','France',SorbonneLoc]

In [44]:
df

,Institution Name,City/Town,Country,Address
0,Massachusetts Institute of Technology (MIT),Cambridge,United States,"77 Massachusetts Avenue Cambridge, United States"
1,Stanford University,Stanford,United States,"450 Serra Mall Stanford, United States"
2,Harvard University,Cambridge,United States,"1350 Massachusetts Avenue Cambridge, United St..."
3,University of Oxford,Oxford,United Kingdom,"Wellington Square Oxford, United Kingdom"
4,California Institute of Technology (Caltech),Pasadena,United States,"1200 East California Boulevard Pasadena, Unite..."
5,ETH Zurich - Swiss Federal Institute of Techno...,Zürich,Switzerland,"Raemistrasse 101 Zürich, Switzerland"
6,University of Cambridge,Cambridge,United Kingdom,"The Old Schools, Trinity Lane Cambridge, Unite..."
7,UCL,London,United Kingdom,"University College London, Gower Street London..."
8,Imperial College London,London,United Kingdom,"South Kensington London, United Kingdom"
9,University of Chicago,Chicago,United States,"Edward H. Levi Hall, 5801 South Ellis Avenue C..."


In [45]:
df.Address[52]

'60 rue Mazarine, Paris, 75006, France'

In [48]:
def get_geocoder(address):
     # initialize your variable to None
    lat_lng_coords = None
     # loop until you get the coordinates
    while(lat_lng_coords is None):
        geoLoc = geocoder.arcgis(address)
        lat_lng_coords = geoLoc.latlng
    latitude = lat_lng_coords[0]
    longitude = lat_lng_coords[1]
    return latitude, longitude

In [49]:
df['Latitude'], df['Longitude'] = zip(*df['Address'].apply(get_geocoder))

In [50]:
df

,Institution Name,City/Town,Country,Address,Latitude,Longitude
0,Massachusetts Institute of Technology (MIT),Cambridge,United States,"77 Massachusetts Avenue Cambridge, United States",42.358917,-71.093461
1,Stanford University,Stanford,United States,"450 Serra Mall Stanford, United States",37.426896,-122.170513
2,Harvard University,Cambridge,United States,"1350 Massachusetts Avenue Cambridge, United St...",42.372648,-71.118657
3,University of Oxford,Oxford,United Kingdom,"Wellington Square Oxford, United Kingdom",51.757367,-1.262548
4,California Institute of Technology (Caltech),Pasadena,United States,"1200 East California Boulevard Pasadena, Unite...",34.139097,-118.123866
5,ETH Zurich - Swiss Federal Institute of Techno...,Zürich,Switzerland,"Raemistrasse 101 Zürich, Switzerland",47.376218,8.548775
6,University of Cambridge,Cambridge,United Kingdom,"The Old Schools, Trinity Lane Cambridge, Unite...",52.206223,0.116372
7,UCL,London,United Kingdom,"University College London, Gower Street London...",51.524600,-0.134800
8,Imperial College London,London,United Kingdom,"South Kensington London, United Kingdom",51.498070,-0.174040
9,University of Chicago,Chicago,United States,"Edward H. Levi Hall, 5801 South Ellis Avenue C...",41.789574,-87.600924


Show the map

In [61]:
# this is coordinates of Niangara, a town in the Democratic Republic of the Congo
# luckily, if we see the map, this town is approximately in the center of all top uni locations
# so, we can use this town coordinates as the epicentrum of our map
lat, lon = 3.685581, 27.876679

In [62]:
# create map of the world-top 100 uni using latitude and longitude values
mapUni = folium.Map(location=[lat, lon], zoom_start=2)

# add markers to map
for lat, lng, name, city, country in zip(df['Latitude'], df['Longitude'], df['Institution Name'], df['City/Town'], df['Country']):
    label = '{}, {}, {}'.format(name, city, country)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mapUni)  
    
mapUni # show the map

[48.8547184612788, 2.337915150816757]